## Обучение embedding модели

In [1]:
# install libs
!pip install -U sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### Проверка CUDA

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu124
True


#### Сборка датасета

In [3]:
import pandas as pd


train_df = pd.read_csv('ostap_train.csv')
test_df = pd.read_csv('ostap_test.csv')


train_df.head()

question  \
0  Как ты проводишь свободное время?   
1      Какой у тебя план на будущее?   
2         Какой у тебя любимый цвет?   
3                Ты веришь в судьбу?   
4            Что ты думаешь о любви?   

                                              answer  
0                  Любителя бьют, а время — не ждет!  
1  План на будущее? Широкие массы миллиардеров уж...  
2  Вы мне в конце концов не художник, не дизайнер...  
3  Кому и кобыла невеста, а я верю только в удачу...  
4  Скоро только кошки родятся, а любовь — это про...

#### Инициализация моделей

In [4]:
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets

model_id = "BAAI/bge-m3"  # Hugging Face model ID
matryoshka_dimensions = [1024, 768, 512, 256, 128, 64]

model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [5]:
corpus = train_df['answer']
queries = train_df['question']

# corpus ids as indexes of list
corpus = dict(zip(map(str, range(len(corpus))), corpus))  # Our corpus (cid => document)
queries = dict(zip(map(str, range(len(queries))), queries))  # Our queries (qid => question)

In [6]:
# --- Step 1: Deduplicate the corpus ---

# This dictionary will help us check if a text has been seen before.
seen_texts = {}

# new_corpus will hold the deduplicated texts with new document IDs.
new_corpus = {}

# This dictionary maps old corpus IDs to the new corpus IDs.
old_to_new = {}

new_id = 0
for old_id, text in corpus.items():
    if text in seen_texts:
        # If this text is a duplicate, map the old ID to the existing new ID.
        old_to_new[old_id] = seen_texts[text]
    else:
        # Otherwise, add the text as a new entry.
        new_id_str = str(new_id)
        new_corpus[new_id_str] = text
        seen_texts[text] = new_id_str
        old_to_new[old_id] = new_id_str
        new_id += 1

print(f"Original corpus size: {len(corpus)}")
print(f"Deduplicated corpus size: {len(new_corpus)}")

# --- Step 2: Update the relevant docs mapping ---

new_relevant_docs = {}
for qid in queries.keys():
    new_relevant_docs[qid] = {old_to_new[qid]}

Original corpus size: 904
Deduplicated corpus size: 861


In [7]:
matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=new_corpus,
        relevant_docs=new_relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,  # Truncate the embeddings to a certain dimension
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)

# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

#### Функция Loss

In [8]:
from sentence_transformers import SentenceTransformerModelCardData, SentenceTransformer


model_id = "BAAI/bge-m3"

# load model with SDPA for using Flash Attention 2
model = SentenceTransformer(
    model_id,
    model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="ru",
        license="apache-2.0",
        model_name="BGE m3 for Ostap project",
    ),
)

In [9]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [1024, 768, 512, 256, 128, 64]  # Important: large to small
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

#### Finetuning

In [10]:
# login to wandb
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ihmatullaev (fitlemon) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [11]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from datasets import Dataset

# dataset from pandas
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="bge-m3-ru-ostap", # output directory and hugging face model ID
    num_train_epochs=4,                         # number of epochs
    per_device_train_batch_size=8,             # train batch size
    gradient_accumulation_steps=1,             # for a global batch size of 512
    per_device_eval_batch_size=8,              # evaluation batch size
    warmup_ratio=0.1,                           # warmup ratio
    learning_rate=2e-5,                         # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",                 # use constant learning rate scheduler
    optim="adamw_torch_fused",                  # use fused adamw optimizer
    tf32=False,                                  # use tf32 precision
    bf16=False,
    fp16=True,# use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="epoch",                      # evaluate after each epoch
    save_strategy="epoch",                      # save after each epoch
    logging_steps=10,                           # log every 10 steps
    save_total_limit=3,                         # save only the last 3 models
    load_best_model_at_end=True,                # load the best model when training ends
    metric_for_best_model="eval_dim_1024_cosine_recall@3",  # Optimizing for the best ndcg@10 score for the 128 dimension
    run_name="bge-m3-ru-ostap",      # name of the run
    report_to="wandb",                    # report to tensorboard
)

In [12]:
from sentence_transformers import SentenceTransformerTrainer

trainer = SentenceTransformerTrainer(
    model=model, # bg-base-en-v1
    args=args,  # training arguments
    train_dataset=train_dataset.select_columns(
        ["question", "answer"]
    ),  # training dataset
    loss=train_loss,
    evaluator=evaluator,
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ihmatullaev (fitlemon) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


TrainOutput(global_step=452, training_loss=1.5578972602312544, metrics={'train_runtime': 502.6187, 'train_samples_per_second': 7.194, 'train_steps_per_second': 0.899, 'total_flos': 0.0, 'train_loss': 1.5578972602312544, 'epoch': 3.991150442477876})

In [ ]:
# save the best model
trainer.save_model()

In [15]:
from google.colab import userdata

In [16]:
# push model to hub
trainer.model.push_to_hub("bge-m3-ru-ostap", exist_ok=True, token=userdata.get('HF_TOKEN'))

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

'https://huggingface.co/fitlemon/bge-m3-ru-ostap/commit/cae49e58712689a895bd21635973db77ae927fb9'

Ссылка на гитлаб: https://colab.research.google.com/drive/1xz83RTeQ9OnpSVHSHhuXoUepk2GuAlcl?usp=sharing